# Text Completion

This example intends to provide a useful code with Llama_index library using a Hugginface pre-trained model to complete the prompt established as introduction for starting the conversation with the model.

The objective of this example was to run a complete functional example in an on-premise GPU of only 12Gb.

### References

* [Tiny Llama Model](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0)
* [Llama 2 Model Description](https://ai.meta.com/resources/models-and-libraries/llama/)

## 1. Code using pytorch and trasformers

In [1]:
import torch
from transformers import pipeline
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

2024-01-19 12:16:55.774974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-19 12:16:55.803408: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 12:16:56.349374: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
print(outputs[0]["generated_text"])

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
I do not have access to specific data on the number of helicopters a human can eat in one sitting. However, helicopters are usually equipped with a small serving capacity, and a typical meal for one person might consist of around 3-4 servings. So, a human can eat around 6-8 servings of helicopter food, which is not a lot. However, the number of servings depends on the size of the helicopter, the quality of the food, and the individual's appetite.


## 1. To Llamaindex

In [2]:
from llama_index import set_global_tokenizer
# tokenizer for huggingface
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0").encode
)

In [1]:
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate


llm = HuggingFaceLLM(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    query_wrapper_prompt=PromptTemplate(
        "<|system|>\n</s>\n<|user|>\n</s>\n<|assistant|>\n"),
    context_window=2048,
    max_new_tokens=256,
    model_kwargs={'trust_remote_code': True},
    generate_kwargs={"temperature": 0.7,"do_sample":True},
    device_map="auto",
)

In [2]:
query_str = "How many helicopters can a human eat in one sitting?"

response = llm.complete(query_str)
print(response.text)

Yes, I can provide you with a list of recommended resources for learning about and practicing mindfulness. Here are some options:

1. Mindfulness-Based Stress Reduction (MBSR) by Jon Kabat-Zinn. This is a comprehensive program that teaches mindfulness meditation and other techniques for managing stress, pain, and other negative emotions.

2. The Mindful Way Workbook by Mark Williams and Danny Penman. This is a step-by-step guide that helps readers develop mindfulness practices and understand the science behind it.

3. Mindfulness-Based Cognitive Therapy (MBCT) by Martin Seligman. This is a popular mindfulness-based approach to treating depression that emphasizes cognitive-behavioral therapy and mindfulness meditation.

4. Mindfulness-Based Relaxation: A Guide to Stress Reduction (Second Edition) by Sharon Begley. This book provides step-by-step instructions for practicing mindfulness meditation and relaxation techniques, as well as scientific research on their effectiveness in reducing

In [3]:
from llama_index.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a friendly chatbot who always responds"
    ),
    ChatMessage(role="user", content=query_str),
]
resp = llm.chat(messages)
print(resp)

2024-01-28 01:37:02.033643: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-28 01:37:02.059520: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 01:37:02.639169: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


assistant: <|assistant|>
I do not have access to specific information about any living being's diet or nutrition. However, according to scientific studies, humans consume approximately 35 grams of energy (calories) per kg of body weight per day. In other words, you would have to consume approximately 9,000 calories (35 grams * 2,400 pounds) to sustain yourself in a single sitting, assuming you were at your full weight.


## GPT Response

On January 23

Humans cannot eat helicopters. Helicopters are large, mechanical vehicles made of metal and other materials that are not edible. Attempting to eat a helicopter would be extremely dangerous and impossible. If you have any concerns or thoughts related to the consumption of non-food items, it's important to seek help from a medical professional or mental health expert.

## Bard Response

On January 23


A human cannot eat a helicopter in one sitting. Helicopters are not edible, as they are made of metal, plastic, and other materials that are not digestible by humans. Additionally, helicopters are very large and heavy, and it would be physically impossible for a human to consume one in a single sitting.

It is important to note that the question of how many helicopters a human can eat in one sitting is a hypothetical one, and it is not something that should be attempted in real life. Ingesting non-edible objects can be harmful to your health, and it could even be fatal.

## GPU- RAM uses after complete the task

In [13]:
import psutil
import GPUtil
import os
# Hugging Face Tokenizers Warning: This warning is related to the parallelism feature in
# Hugging Face’s tokenizers library. When a process that has already used parallelism
# gets forked, it can lead to deadlocks, which is why parallelism is disabled.

# Avoid using tokenizers before the fork if possible.
os.environ["TOKENIZERS_PARALLELISM"] = "false"


gpus = GPUtil.getGPUs()


ram_info = psutil.virtual_memory()

print(f"Total RAM: {ram_info.total / (1024 ** 3):.2f} GB")
print(f"Aviable RAM: {ram_info.available / (1024 ** 3):.2f} GB")
print(f"Used RAM: {ram_info.used / (1024 ** 3):.2f} GB\n")
if gpus:
    print(f"Total GPU: {gpus[0].memoryTotal} MB")

    # Imprimir la cantidad de memoria GPU disponible
    print(f"Aviable GPU: {gpus[0].memoryFree} MB")
    print(f"Usage GPU: {gpus[0].memoryUsed} MB")
else:
    print("There aren't aviable GPU.")

Total RAM: 31.24 GB
Aviable RAM: 28.35 GB
Used RAM: 2.39 GB

Total GPU: 12288.0 MB
Aviable GPU: 7641.0 MB
Usage GPU: 4403.0 MB
